In [1]:
import json
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.metrics import f1_score

from sys import path

path.append('../utils/')

from utils import get_datasets

from utils import load_preds

from scipy.stats import t as table_t

In [2]:
DATASETS = ["20ng", "acm", "webkb", "reut"]
DATASETS = ["acm", "20ng"]

In [3]:
pd_datasets = get_datasets(DATASETS)

In [8]:
st1 = "calibrated"

st2s = [
    f"local_rf/zero_train_False/{alfa}/bert_temperature_scaling/ktmk_isotonic/xlnet_temperature_scaling/dist_probas_probas_based"
    for alfa in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
]

#st2s = ["calibrated"]

fold_values = {}
for dataset in DATASETS:
    fold_values[dataset] = {}
    for st2 in st2s:
        fold_values[dataset][st2] = []
        diffs = []
        alpha = 0.05 / 2
        for fold in np.arange(10):

            ppath = f"/home/welton/data/stacking/stacking_output/{dataset}/10_folds/logistic_regression/{st1}/fold_{fold}/scoring.json"
            with open(ppath, 'r') as fd:
                st1_macro = json.load(fd)["f1_macro"]
            
            mpath = f"/home/welton/data/stacking/stacking_output/{dataset}/10_folds/logistic_regression/{st2}/fold_{fold}/scoring.json"
            with open(mpath, 'r') as fd:
                st2_macro = json.load(fd)["f1_macro"]
            
            diffs.append(st2_macro - st1_macro)
            fold_values[dataset][st2].append([st1_macro, st2_macro])
        #    print(st2_macro, st1_macro, st2_macro - st1_macro)
            
        mean = np.mean(diffs)
        #print(mean)
        std = np.std(diffs) / np.sqrt(len(diffs))

        t = table_t.ppf(1 - alpha, df=9)
        ground = np.round(mean - t * std, decimals=4)
        ceiling = np.round(mean + t * std, decimals=4)

        print(f"{dataset.upper()} - ({ground} - {ceiling})")
        #print("*"*100)

ACM - (-0.0382 - -0.0093)
ACM - (-0.0274 - -0.0056)
ACM - (-0.0235 - -0.0017)
ACM - (-0.012 - 0.0047)
ACM - (-0.0097 - 0.0104)
ACM - (-0.0083 - 0.0084)
ACM - (-0.0026 - 0.0056)
ACM - (-0.0002 - 0.0019)
ACM - (-0.0001 - 0.0004)
ACM - (-0.0 - 0.0)
20NG - (-0.06 - -0.0462)
20NG - (-0.0381 - -0.0286)
20NG - (-0.0331 - -0.0235)
20NG - (-0.0176 - -0.0108)
20NG - (-0.0135 - -0.0077)
20NG - (-0.0133 - -0.0063)
20NG - (-0.0066 - -0.0031)
20NG - (-0.0022 - 0.0026)
20NG - (-0.0012 - 0.0)
20NG - (-0.0017 - 0.001)


In [9]:
for dataset in DATASETS:
    for st2 in st2s:
        df = pd.DataFrame(fold_values[dataset][st2], columns=["st1", "st2"])
        mrep = np.round(np.mean(df.st1.values) * 100, decimals=2)
        prob = np.round(np.mean(df.st2.values) * 100, decimals=2)
        print(f"{dataset.upper()} - {st1}: {mrep} {st2}: {prob}")

ACM - calibrated: 73.95 local_rf/zero_train_False/0.1/bert_temperature_scaling/ktmk_isotonic/xlnet_temperature_scaling/dist_probas_probas_based: 71.58
ACM - calibrated: 73.95 local_rf/zero_train_False/0.2/bert_temperature_scaling/ktmk_isotonic/xlnet_temperature_scaling/dist_probas_probas_based: 72.3
ACM - calibrated: 73.95 local_rf/zero_train_False/0.3/bert_temperature_scaling/ktmk_isotonic/xlnet_temperature_scaling/dist_probas_probas_based: 72.69
ACM - calibrated: 73.95 local_rf/zero_train_False/0.4/bert_temperature_scaling/ktmk_isotonic/xlnet_temperature_scaling/dist_probas_probas_based: 73.58
ACM - calibrated: 73.95 local_rf/zero_train_False/0.5/bert_temperature_scaling/ktmk_isotonic/xlnet_temperature_scaling/dist_probas_probas_based: 73.99
ACM - calibrated: 73.95 local_rf/zero_train_False/0.6/bert_temperature_scaling/ktmk_isotonic/xlnet_temperature_scaling/dist_probas_probas_based: 73.96
ACM - calibrated: 73.95 local_rf/zero_train_False/0.7/bert_temperature_scaling/ktmk_isotonic/xl

In [25]:
np.load("/home/welton/data/oracle/local_rf/isotonic/acm/10_folds/ktmk/dist_probas_probas_based/0/train.npz")['y']

array([0., 1., 1., ..., 0., 1., 0.])